<a href="https://www.kaggle.com/code/pgvishnu526/prompt2manga?scriptVersionId=282190056" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 📚 Prompt2Manga – Multi-Agent Manga Generation with Gemini

## 1. Problem Statement

Creating a short manga or comic usually requires:
- Character design
- Story writing
- Dialog scripting
- Panel-wise illustration
- Layout and publishing

Doing this manually is time-consuming and requires multiple skills (writer, artist, designer).

## 2. Solution Overview

**Prompt2Manga** is an AI-powered, **Sequential Multi-Agent System** that converts a simple user prompt into a **5-page manga PDF**.

Given a user prompt (e.g., *"Create a 5-page manga about a robot who learns human emotions"*), the system:

1. Generates characters and their traits  
2. Builds a story outline  
3. Converts the story into a panel-wise script  
4. Generates images using a diffusion model  
5. Compiles everything into a final manga PDF

All of this runs inside a **single Kaggle notebook**.

---

## 3. Key Features (Course Concepts Used)

This project demonstrates multiple agentic / GenAI concepts:

- ✅ **Sequential Multi-Agent Pipeline**  
  Each agent focuses on one task and passes structured output to the next agent.

- ✅ **LLM-Orchestrated Workflow**  
  Gemini (or LLM) is used to:
  - Generate characters
  - Plan the story
  - Write dialog/script

- ✅ **Tool-Using Agent**  
  One sub-agent triggers image generation using a **Stable Diffusion pipeline** (tool use).

- ✅ **Automated Document Generation**  
  A final “publisher agent” composes images and text into a **multi-page manga PDF**.

---

## 4. High-Level Architecture

The system has the following agents:

1. **Character Agent**  
   - Input: User prompt  
   - Output: List of characters with names, roles, and personality traits.

2. **Story Agent**  
   - Input: User prompt + character list  
   - Output: Story outline divided into pages and panels.

3. **Script Agent**  
   - Input: Story outline  
   - Output: Panel-wise script including dialogues and scene descriptions.

4. **Illustration Agent**  
   - Input: Panel descriptions  
   - Output: Panel images generated using Stable Diffusion.

5. **Publisher Agent**  
   - Input: Script + images  
   - Output: A **5-page PDF** saved as `final_manga_comic.pdf`.

Data flows in a **linear sequence**:  
**User Prompt → Character Agent → Story Agent → Script Agent → Illustration Agent → Publisher Agent**.

---

## 5. Architecture Diagram (Logical View)

You can visualize the system like this:

**User Prompt**  
⬇  
**Character Agent** → Characters JSON  
⬇  
**Story Agent** → Story Outline (pages + panels)  
⬇  
**Script Agent** → Panel Script (dialog + descriptions)  
⬇  
**Illustration Agent** → Panel Images  
⬇  
**Publisher Agent** → 📄 `final_manga_comic.pdf`

---

## 6. Technologies Used

- **Language Model**: Gemini (or compatible LLM)
- **Image Generation**: Stable Diffusion (Diffusers)
- **PDF Creation**: FPDF
- **Runtime**: Kaggle Notebook (Python)
- **Hardware**: GPU (T4) for faster image generation

---

## 7. How to Run (Kaggle Setup)

1. **Enable GPU**
   - Go to **Settings → Accelerator → GPU (T4)**

2. **Install Dependencies** (run the setup cell)
   - `diffusers`, `transformers`, `accelerate`, `fpdf`, etc.

3. **Run All Cells**
   - From the top menu: **Run → Run All**
   - Or run each section in order:
     1. Imports & Setup  
     2. Model Loading  
     3. Agent Definitions  
     4. Manga Generation Pipeline  
     5. PDF Creation

4. **Output Location**
   - The final manga is saved at:
     ```bash
     /kaggle/working/final_manga_comic.pdf
     ```

---

## 8. File Outputs

- `final_manga_comic.pdf` – 5-page manga generated from user prompt  
- Individual panel images (PNG/JPG) stored in `/kaggle/working/`

---

## 9. Limitations & Future Work

- Image generation speed depends on GPU.
- Currently supports a fixed number of pages (e.g., 5).
- Future improvements:
  - Interactive UI for prompt input
  - Parameter control for style (chibi, realistic, etc.)
  - Multi-language support for dialogues


In [1]:
!pip install "protobuf==3.20.3" diffusers transformers accelerate safetensors fpdf2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**SETUP**

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

print("✅ API Key successfully loaded into Environment.")

✅ API Key successfully loaded into Environment.


**Creating tool for our use case**

In [3]:
import json
import torch
import time
from fpdf import FPDF
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# ============================================================
# 1. MODEL SETUP – Stable Diffusion (Anything V5 for Manga/Anime)
# ============================================================

print("🔄 Loading Anime Model...")
try:
    # "Anything V5" is a Stable Diffusion variant tuned for anime/manga style artwork
    model_id = "stablediffusionapi/anything-v5"
    
    # Load the diffusion pipeline in half precision (float16) on GPU
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16
    ).to("cuda")
    
    # Use an efficient scheduler for faster and cleaner images
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    
    # Disable the safety checker to allow action scenes / intense visuals
    pipe.safety_checker = None
    pipe.requires_safety_checker = False
    
    print("✅ Model Loaded & Safety Checker Disabled (Action Scenes Allowed).")
except Exception as e:
    # If model load fails, we keep pipe=None so later tools can fall back gracefully
    print(f"⚠️ Model Load Error: {e}")
    pipe = None


# ============================================================
# 2. GLOBAL PATHS – Where we store images, script, and state
# ============================================================

WORK_DIR   = "/kaggle/working"
IMG_DIR    = os.path.join(WORK_DIR, "images")          # All generated panel images
SCRIPT_FILE = os.path.join(WORK_DIR, "manga_script.json")  # Script from the LLM/agent
STATE_FILE  = os.path.join(WORK_DIR, "manga_state.json")   # Script + image paths for PDF

os.makedirs(IMG_DIR, exist_ok=True)


# ============================================================
# 3. HELPER – Fix encoding issues for PDF text
# ============================================================

def sanitize_text(text):
    """
    Cleans Unicode punctuation (like smart quotes, em-dash, ellipsis)
    and converts text to a PDF-safe Latin-1 representation.
    """
    if not isinstance(text, str):
        return str(text)
    
    replacements = {
        "\u2026": "...",  # ellipsis
        "\u2018": "'",    # left single quote
        "\u2019": "'",    # right single quote
        "\u201c": '"',    # left double quote
        "\u201d": '"',    # right double quote
        "\u2013": "-",    # en dash
        "\u2014": "-"     # em dash
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    
    # Ensure the final string is Latin-1 compatible for FPDF
    return text.encode('latin-1', 'replace').decode('latin-1')


# ============================================================
# 4. TOOL 1 – Save Script from Agent
# ============================================================

def save_script_tool(json_input: str):
    """
    TOOL 1 – Accepts a JSON string from the LLM/agent, 
    cleans it, parses it, and saves to SCRIPT_FILE.
    
    Input : json_input (string, may contain ```json ... ``` wrapper)
    Output: Status string ("SCRIPT_SAVED" or error message)
    """
    try:
        # Remove common Markdown fencing like ```json ... ```
        clean = json_input.replace("```json", "").replace("```", "").strip()
        
        # Parse JSON into Python structure
        data = json.loads(clean)
        
        # Save pretty-printed JSON for easier debugging
        with open(SCRIPT_FILE, 'w') as f:
            json.dump(data, f, indent=2)
        
        return "SCRIPT_SAVED"
    except Exception as e:
        return f"Save Error: {e}"


# ============================================================
# 5. TOOL 2 – Generate Images from Script
# ============================================================

def generate_images_tool(_ignored_input: str):
    """
    TOOL 2 – Reads the saved script and generates panel images.
    
    - Reads SCRIPT_FILE
    - For each scene:
        - Build a Stable Diffusion prompt from description + mood
        - Generate an image (if model is loaded)
        - Attach image path back to the scene
    - Saves merged result to STATE_FILE
    
    Input : (unused, required by tool interface)
    Output: Status string ("IMAGES_GENERATED" or error message)
    """
    try:
        if not os.path.exists(SCRIPT_FILE):
            return "Error: No script file."
        
        # Load script (could be dict or nested JSON string)
        with open(SCRIPT_FILE, 'r') as f:
            data = json.load(f)

        if isinstance(data, str):
            data = json.loads(data)
            
        scenes = data.get('scenes', [])
        updated_scenes = []
        
        print(f"🎨 Generating {len(scenes)} Panels...")
        
        for i, scene in enumerate(scenes):
            # Ensure scene is always a dict for safety
            if isinstance(scene, str):
                scene = json.loads(scene)
                
            desc = scene.get('description', '')
            mood = scene.get('mood', '')
            img_path = os.path.join(IMG_DIR, f"scene_{i+1}.png")
            
            if pipe:
                # Positive prompt: emphasize manga style, clean art
                prompt = (
                    "masterpiece, best quality, manga style, monochrome, greyscale, lineart, "
                    f"{desc}, {mood}, intense action, highly detailed, 4k"
                )
                
                # Negative prompt: avoid unwanted artifacts
                negative = (
                    "color, 3d, realistic, blurry, messy, sketch, text, watermark, "
                    "bad anatomy, deformed"
                )
                
                # Use autocast for faster, memory-efficient generation
                with torch.autocast("cuda"):
                    image = pipe(
                        prompt,
                        negative_prompt=negative,
                        num_inference_steps=25,  # speed/quality trade-off
                        width=512,
                        height=768,
                        guidance_scale=8.0
                    ).images[0]
                
                image.save(img_path)
                scene['image_path'] = img_path
            else:
                # Fallback if model failed to load; keeps pipeline alive for testing
                scene['image_path'] = "dummy.png"
            
            updated_scenes.append(scene)
        
        # Save combined state (script + image paths)
        with open(STATE_FILE, 'w') as f:
            json.dump({"scenes": updated_scenes}, f, indent=2)
            
        return "IMAGES_GENERATED"
    except Exception as e:
        return f"Gen Error: {e}"


# ============================================================
# 6. TOOL 3 – Create Manga-Style PDF from Images + Script
# ============================================================

def create_pdf_tool(_ignored_input: str):
    """
    TOOL 3 – Uses STATE_FILE (scenes + image paths) to create a manga PDF.
    
    Layout:
    - A4 portrait
    - 2 scenes (panels) per page: top and bottom
    - Each panel: image + narration box + dialogue box
    
    Input : (unused, required by tool interface)
    Output: Status string with PDF path, or error message
    """
    try:
        if not os.path.exists(STATE_FILE):
            return "Error: No state file."
        
        # ---- 1. Load and robustly parse state JSON ----
        with open(STATE_FILE, 'r') as f:
            raw_data = f.read().strip()
        
        data = None
        data_str = raw_data
        
        # Try multiple times in case of nested JSON strings
        for _ in range(5):
            try:
                data = json.loads(data_str)
                if not isinstance(data, str):
                    break  # parsed into final structure (dict)
                data_str = data
            except:
                break
        
        scenes = data.get('scenes', []) if isinstance(data, dict) else []
        
        # ---- 2. Setup PDF (A4) ----
        pdf = FPDF(orientation='P', unit='mm', format='A4')
        pdf.set_auto_page_break(auto=False)  # manual layout
        
        # Helper: draw narration/dialog boxes on top of the panel image
        def draw_text_box(pdf, text, x, y, w, h, is_dialogue=False):
            if not text:
                return
            
            text = sanitize_text(text)
            
            # Dialogue: white bubble, Narration: light grey box
            if is_dialogue:
                pdf.set_fill_color(255, 255, 255)    # white
            else:
                pdf.set_fill_color(240, 240, 240)    # light grey
                
            pdf.set_draw_color(0, 0, 0)   # black border
            pdf.set_line_width(0.3)
            
            # Draw the filled rectangle (box)
            pdf.rect(x, y, w, h, style='FD')
            
            # Text properties
            pdf.set_xy(x + 2, y + 2)
            pdf.set_text_color(0, 0, 0)
            font_size = 10 if is_dialogue else 9
            font_style = 'B' if is_dialogue else 'I'
            pdf.set_font("Arial", font_style, font_size)
            
            # Wrap text inside the box
            pdf.multi_cell(w - 4, 5, text, align='C' if is_dialogue else 'L')

        # ---- 3. Layout each scene as a panel ----
        for i, scene in enumerate(scenes):
            if not isinstance(scene, dict):
                continue

            # New page for every 2 scenes
            if i % 2 == 0:
                pdf.add_page()
                
                # Optional title only on the first page
                if i == 0:
                    pdf.set_font("Arial", "B", 16)
                    pdf.cell(0, 10, "AI Generated Manga", ln=True, align='C')

            # Top or bottom half of the page
            base_y = 20 if (i == 0) else 10
            y = base_y if (i % 2 == 0) else 148  # 2 panels per page
            
            img_path = scene.get('image_path', '')
            x_img = 10
            w_img = 190
            h_img = 130
            
            # Draw panel image with a border
            if os.path.exists(img_path):
                pdf.image(img_path, x=x_img, y=y, w=w_img, h=h_img)
                pdf.set_draw_color(0, 0, 0)
                pdf.set_line_width(1.0)
                pdf.rect(x_img, y, w_img, h_img)

            # Short narration box (top-left)
            narrative = scene.get('description', '')[:100]
            if narrative:
                draw_text_box(
                    pdf, narrative,
                    x=x_img + 2, y=y + 2,
                    w=80, h=20,
                    is_dialogue=False
                )
            
            # Dialogue: can be list of lines or a single string
            dialogue = scene.get('dialogue')
            dialogue_text = ""
            
            if isinstance(dialogue, list):
                for d in dialogue:
                    line = d.get('line', '') if isinstance(d, dict) else str(d)
                    dialogue_text += line + " "
            elif isinstance(dialogue, str):
                dialogue_text = dialogue
            
            # Dialogue bubble (bottom-right)
            if len(dialogue_text) > 0:
                # Adjust height based on approximate length
                box_h = 20 + (len(dialogue_text) // 30) * 5
                draw_text_box(
                    pdf, dialogue_text,
                    x=x_img + 110,
                    y=y + h_img - box_h - 5,
                    w=75, h=box_h,
                    is_dialogue=True
                )
        
        # ---- 4. Save final PDF ----
        out_path = "/kaggle/working/Final_Manga_Comic.pdf"
        pdf.output(out_path)
        
        return f"PDF_CREATED: {out_path}"
        
    except Exception as e:
        return f"PDF Error: {e}"


2025-11-27 12:02:19.135260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764244939.327802      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764244939.378991      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔄 Loading Anime Model...


model_index.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


✅ Model Loaded & Safety Checker Disabled (Action Scenes Allowed).


In [4]:
!pip install -U google-genai google-adk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 148.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 376.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 354.6 kB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.48.0
    Uninstalling google-genai-1.48.0:
      Successfully uninstalled google-genai-1.48.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 1.18.0
    U

## 🧬 Detailed Architecture & Data Flow

This notebook follows a **Sequential Agent Architecture**:

1. **Input Layer**
   - A single user text prompt:
     - Example: *"Create a 5-page manga about a shy girl who becomes a hero."*

2. **Agent 1 – Character Agent**
   - Uses Gemini/LLM to:
     - Identify main and side characters.
     - Define names, traits, roles (protagonist, antagonist, etc.).
   - Output format: **JSON-like structure** (characters list).

3. **Agent 2 – Story Agent**
   - Takes user prompt + character list.
   - Produces a **page-wise story outline**:
     - Page 1 → Introduction
     - Page 2–4 → Conflict & development
     - Page 5 → Resolution

4. **Agent 3 – Script Agent**
   - Converts the outline into a **panel-wise script**:
     - For each panel:
       - Scene description
       - Characters present
       - Dialogues

5. **Agent 4 – Illustration Agent**
   - For each panel description:
     - Builds a Stable Diffusion prompt.
     - Calls the diffusion pipeline to generate an image.
   - Saves images as `panel_1.png`, `panel_2.png`, etc.

6. **Agent 5 – Publisher Agent**
   - Uses FPDF to:
     - Create a PDF.
     - Place panel images and dialogues on each page.
   - Final output: `final_manga_comic.pdf`.

---

## 🔁 Control Flow

1. Initialize all agents.
2. Pass the user prompt to **Character Agent**.
3. Pass characters → **Story Agent**.
4. Pass story → **Script Agent**.
5. Pass script → **Illustration Agent**.
6. Pass script + images → **Publisher Agent**.
7. Show final path of the PDF in the notebook output.


In [5]:
from google.adk.agents import LlmAgent,SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  
    exp_base=7, 
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], 
)

In [7]:
config_model = Gemini(
    model="gemini-2.5-flash-lite", 
    retry_options=retry_config
)

In [8]:
# 1. CHARACTER AGENT

char_agent = LlmAgent(
    name ="character_agent",
    model = config_model,
    instruction="""
        Role: Lead Character Designer.
        Task: Define the main protagonist for a Manga.
        Output Requirement: strictly valid JSON with keys:
        - "name": Character name.
        - "type": EXACTLY one of: ["Person", "Animal", "Fruit", "Object"].
        - "appearance": A dense, comma-separated string of visual keywords matching the 'type'.
          (e.g., "cybernetic wolf, metallic fur, glowing red eyes, forest background").
        - "personality": Brief archetype.
        
        NO CHAT. ONLY JSON.
        """,
    output_key = "char_data"
    
)

In [9]:
# 2. STORY AGENT

story_agent = LlmAgent(
    name="story_agent",
    model=config_model,
    instruction="""
        Role: Manga Director.
        Input: {char_data}
        Task: Create a detailed **10-panel story sequence** (approx 5 pages).
        
        Structure the pacing carefully:
        - Scenes 1-2: Introduction (Establish the setting and the character).
        - Scenes 3-4: The Conflict (An enemy appears or a problem starts).
        - Scenes 5-8: The Action/Climax (Dynamic battles, chases, or intensity).
        - Scenes 9-10: Resolution (The aftermath and cool ending pose).

        Output Requirement: strictly valid JSON with key "scenes", containing a list of 10 objects. Each object must have:
        - "scene_index": The number (1-10).
        - "description": Detailed visual instructions for the artist. **Focus on ACTION and CAMERA ANGLES** (e.g., "Low angle shot," "Close up on eyes," "Wide shot of explosion").
        - "mood": The atmosphere (e.g., "Tense", "Victory", "Dark").
        
        **CRITICAL SAFETY**: Describe action with 'energy', 'impact', 'motion blur'. Do NOT use 'blood' or 'gore'.
        NO CHAT. ONLY JSON.
        """,
    output_key = "story_data"
)

In [10]:
# 3. SCRIPT AGENT( Uses Tool)

script_agent = LlmAgent(
    name="dialogue_agent",
    model=config_model,
    instruction="""
        Role: Manga Script Writer.
        Input: {story_data}
        Task: 
        1. Review the full 10-scene sequence to understand the flow.
        2. Add 'dialogue' to EACH scene. 
           - Early scenes: Set the mystery.
           - Middle scenes: Short, punchy shouts or sound effects (SFX).
           - Final scenes: A cool one-liner.
        3. CALL `save_script_tool` with the full JSON.
        4. Output the tool result.
        
        Constraint: Keep dialogue under 20 words per bubble.
        NO CHAT.
        """,
    output_key = "save_status",
    tools=[save_script_tool] 
)

In [11]:
# 4. ILLUSTRATOR AGENT (Uses Tool)

illustrator_agent = LlmAgent(
    name="illustrator_agent",
    model=config_model,
    instruction="""
        Role: Render Engine Trigger.
        Task: The script is ready. Initiate the visual pipeline.
        Action: CALL `generate_images_tool` with argument 'start'.
        Do not explain. JUST CALL THE TOOL.
        """,
    output_key = "illus_status",
    tools=[generate_images_tool] 
)

In [12]:
# 5. PUBLISHER AGENT (Uses Tool)

publisher_agent = LlmAgent(
    name="publisher_agent",
    model=config_model,
     instruction="""
        Role: Publisher.
        Task: CALL `create_pdf_tool` with argument 'start'.
        Output the filename.
        NO CHAT.
        """,
    description="Compiles the PDF.",
    tools=[create_pdf_tool] 
)

In [13]:
# ROOT AGENT (Sequential Pattern)

root_agent = SequentialAgent(
    name="root",
    sub_agents=[
        char_agent,   
        story_agent,      
        script_agent,     
        illustrator_agent,  
        publisher_agent     
    ],
    
    description="Robust Manga Pipeline"
)

In [14]:
from google.adk.runners import InMemoryRunner 

runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "A boy going to forest with his grandpa and cousin , they stay nigth there and suddenly a meteor falling on earth near the place they staying and boy run to say it and goes close to say that a watch cameout form the box  and pasted on his hand and suddenly went to alien transformation"
)



 ### Created new session: debug_session_id

User > A boy going to forest with his grandpa and cousin , they stay nigth there and suddenly a meteor falling on earth near the place they staying and boy run to say it and goes close to say that a watch cameout form the box  and pasted on his hand and suddenly went to alien transformation
character_agent > ```json
{
  "name": "Ren",
  "type": "Person",
  "appearance": "young boy, messy dark hair, curious eyes, simple t-shirt, worn shorts, forest setting, glowing alien watch on wrist, subtle alien features emerging",
  "personality": "Adventurous, impulsive, brave"
}
```
story_agent > ```json
{
  "scenes": [
    {
      "scene_index": 1,
      "description": "WIDE SHOT establishing a dense, sun-dappled forest. Ren, a young boy with messy dark hair and curious eyes, walks ahead of an older man (Grandpa) and another child (Cousin). Ren is wearing a simple t-shirt and worn shorts. He carries a small backpack. The mood is peaceful and adventuro

dialogue_agent > The provided JSON already outlines the scene descriptions and moods. I will now add dialogue to each scene, keeping it concise and fitting the established mood and progression of the story.

Scene 1: "Race you to the big oak, Grandpa!" "Hurry up, slowpoke!"
Scene 2: "What is this watch... it feels warm."
Scene 3: "Whoa! What was that?!"
Scene 4: "Gotta see! Gotta see!"
Scene 5: "A... ship?" "*Hiss...*"
Scene 6: "It's... pulling me!" "Bzzt! Snap!"
Scene 7: "AAGH! It burns!"
Scene 8: "*ROAR!*" "Feel the power!"
Scene 9: "What... have I become?"
Scene 10: "This is just the beginning."

Now I will save the script with the added dialogue.


Token indices sequence length is longer than the specified maximum sequence length for this model (101 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['mood is peaceful and adventurous. camera follows from behind, slightly low., adventurous, intense action, highly detailed, 4 k']


🎨 Generating 10 Panels...


  0%|          | 0/25 [00:00<?, ?it/s]

...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['ted by the noise., awestruck, intense action, highly detailed, 4 k']


  0%|          | 0/25 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['attention on ren., confident, intense action, highly detailed, 4 k']


  0%|          | 0/25 [00:00<?, ?it/s]

/tmp/ipykernel_20/3640443298.py:271: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 16)
/tmp/ipykernel_20/3640443298.py:272: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, "AI Generated Manga", ln=True, align='C')
/tmp/ipykernel_20/3640443298.py:255: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", font_style, font_size)
